In [1]:
using Pkg

In [2]:
using CSV, DataFrames, JuMP, Gurobi, PyCall, LinearAlgebra, Ipopt
using JuMP, Juniper, Ipopt, Statistics

In [15]:
precincts = Matrix(CSV.read("precinct_coords.csv", DataFrame, header=false));

In [17]:
hospitals = Matrix(CSV.read("UPDATEDHospitals.csv", DataFrame, header=false)[2:end, 2:end]); 

In [50]:
function haversine_distance(coord1, coord2)
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = deg2rad(coord1[1]), deg2rad(coord1[2])
    lat2, lon2 = deg2rad(coord2[1]), deg2rad(coord2[2])

    # Calculate the differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)^2 + cos(lat1) * cos(lat2) * sin(dlon / 2)^2
    c = 2 * atan(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    dist = R * c

    return dist
end

# Helper function to convert degrees to radians
function deg2rad(deg)
    return deg * π / 180
end

# Example usage:
coord1 = (37.7749, -122.4194)  # Latitude and longitude for San Francisco, CA
coord2 = (34.0522, -118.2437)  # Latitude and longitude for Los Angeles, CA

dist = haversine_distance(coord1, coord2)
println("The distance between the two locations is $dist km.")

The distance between the two locations is 559.1205770615527 km.


In [30]:
census = Matrix(CSV.read("census_data.csv", DataFrame, header=true));

In [62]:
precincts_dict = Dict()
for i in 1:275
    precinct_num = precincts[i, 1]
    location = precincts[i, 2:3]
    precincts_dict[precinct_num] = location
end

In [66]:
function distance_hospi_precj(i,j)
    l1 = hospitals[i, 1:2]
    l2 = precincts_dict[j]
    return haversine_distance(l1,l2)
end

distance_hospi_precj (generic function with 1 method)

In [81]:
function distance_hospi_personj(i,j)
    l1 = hospitals[i, 1:2]
    l2 = precincts[j, 2:3]
    return haversine_distance(l1,l2)
end

distance_hospi_personj (generic function with 1 method)

In [87]:
population

Dict{Any, Any} with 275 entries:
  1813.0 => [1636, 270, 791, 486, 24, 8, 2, 55]
  1107.0 => [2307, 1064, 406, 609, 145, 15, 2, 66]
  2108.0 => [3255, 1873, 103, 296, 855, 20, 0, 108]
  2004.0 => [1657, 1278, 82, 157, 86, 7, 0, 47]
  308.0  => [5588, 2235, 298, 482, 2490, 7, 3, 73]
  1703.0 => [2933, 374, 1705, 515, 137, 15, 0, 187]
  115.0  => [0, 0, 0, 0, 0, 0, 0, 0]
  1503.0 => [1307, 133, 511, 292, 54, 4, 0, 313]
  112.0  => [2492, 1154, 80, 1004, 147, 2, 0, 105]
  1812.0 => [2367, 657, 941, 607, 52, 21, 1, 88]
  2013.0 => [1169, 1022, 25, 55, 39, 3, 0, 25]
  2015.0 => [2580, 1695, 193, 311, 286, 13, 8, 74]
  404.0  => [2360, 1513, 363, 214, 202, 5, 2, 61]
  905.0  => [2272, 434, 981, 539, 222, 11, 0, 85]
  612.0  => [3322, 2518, 133, 188, 353, 6, 1, 123]
  1704.0 => [2952, 632, 1444, 343, 309, 10, 1, 213]
  2211.0 => [1663, 1096, 63, 144, 271, 7, 1, 81]
  1901.0 => [1760, 1249, 99, 216, 136, 15, 0, 45]
  207.0  => [1760, 1273, 85, 135, 220, 15, 0, 32]
  506.0  => [2500, 1905, 123,

In [86]:
population = Dict()
for i in 1:275
    population[precincts[i,1]] = census[i, 2:end]
end

In [32]:
function classic_distance_model()
    ipopt = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
    optimizer = optimizer_with_attributes(Juniper.Optimizer, "nl_solver"=>ipopt)
    model_a = Model(optimizer)
    facilities_index = 1:10 #we use i for this indexing
    clients_index = 1:1000 #we use j for this indexing
    @variable(model_a, x[facilities_index, clients_index], Bin)
    @variable(model_a, y[facilities_index], Bin)
    @constraint(model_a, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1)
    @constraint(model_a, [i in facilities_index, j in clients_index], x[i,j] <= y[i])
    # facilities_cost = sum(c[i]*y[i] for i in facilities_index) # need to find c
    facilities_cost = 0
    equity_cost = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in clients_index)
    @objective(model_a, Min, facilities_cost+equity_cost);
    return model_a
end

classic_distance_model (generic function with 1 method)

In [ ]:
# african_indices
# american_indices

$$d(P_i) := \frac{1}{|P_i|}\sum_{j \in P_i} d(j, \pi(j)),$$

$d(j, pi(j)) = \sum_{i} x_{i,j}d(i,j)$

In [121]:
function lp_distance_model(p, precincts, population, hospitals)

    #set_up
    ipopt = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
    optimizer = optimizer_with_attributes(Juniper.Optimizer, "nl_solver"=>ipopt)
    model_a = Model(optimizer)

    #indices
    facilities_index = 1:size(hospitals)[1] #we use i for this indexing
    clients_index = 1:length(population) #we use j for this indexing


    #whether client is matched to facility
    @variable(model_a, x[facilities_index, clients_index] >= 0)

    #whether facility is built/operated
    @variable(model_a, y[facilities_index], Bin)

    #every client gets matched to some service provider
    @constraint(model_a, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1)

    # you can only match client to an operating service provider
    @constraint(model_a, [i in facilities_index, j in clients_index], x[i,j] <= y[i])
    println("variables, costs added")


    #CHANGE THIS TO INCLUDE THE COSTS
    # facilities_cost = sum(c[i]*y[i] for i in facilities_index) 
    facilities_cost = 0
    value = []
    equity_cost = 0
    for race in 1:8
        pop = sum(population[precinct[1]][race] for precinct in population)
        dist = sum(sum(sum(x[i,j]*distance_hospi_personj(i,j) for i in facilities_index) for j in clients_index) for precinct in population)
        equity_cost += (dist/pop)^p
    end 
    @NLobjective(model_a, Min, facilities_cost+equity_cost);
    return model_a
end

lp_distance_model (generic function with 2 methods)

In [122]:
model = lp_distance_model(2,precincts, population, hospitals);

variables, costs added


LoadError: InterruptException:

In [ ]:
optimize!(model)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 10010
#IntBinVar: 10010
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 25005.882311105728
Relaxation Obj: 0.0008000272097948911

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                          0.0                -   59966.0     0         -     


[ Info: Breaking out of strong branching as the time limit of 100.0 seconds got reached.


    2       3                 -                          0.0                -   73916.9     -         >>    


In [71]:
@variable(model_a, x[facilities_index, clients_index], Bin);

In [72]:
@variable(model_a, y[facilities_index], Bin);

In [73]:
@constraint(model_a, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1);

In [74]:
@constraint(model_a, [i in facilities_index, j in clients_index], x[i,j] <= y[i]);

In [75]:
facilities_cost = 0

0

In [76]:
function d(i,j)
    return 1
end

d (generic function with 1 method)

In [79]:
# @NLexpression(model, obj_expr, sum(x[i] for i in 1:3))
# @NLobjective(model, Min, obj_expr)
equity_cost = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in clients_index)
@objective(model_a, Min, facilities_cost+equity_cost);

┌ Warning: The addition operator has been used on JuMP expressions a large number of times. This warning is safe to ignore but may indicate that model generation is slower than necessary. For performance reasons, you should not add expressions in a loop. Instead of x += y, use add_to_expression!(x,y) to modify x in place. If y is a single variable, you may also use add_to_expression!(x, coef, y) for x += coef*y.
└ @ JuMP ~/.julia/packages/JuMP/OUdu2/src/operators.jl:279


In [81]:
optimize!(model_a)

nl_solver         : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
feasibility_pump  : false
log_levels        : [:Options, :Table, :Info]

#Variables: 100010
#IntBinVar: 100010
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 5.699100971221924
Relaxation Obj: 9.999999925032196

 ONodes   CLevel          Incumbent                   BestBound            Gap    Time   Restarts  GainGap  
    2       2                 -                          10.0               -    100.8      0         -     


[ Info: Breaking out of strong branching as the time limit of 100.0 seconds got reached.


    2       3                 -                          10.0               -    110.3      -         >>    
    2       4                 -                          10.0               -    119.8      -         >>    
    2       5                 -                          10.0               -    129.4      -         >>    


LoadError: InterruptException:

In [51]:
using JuMP
using Ipopt

function create_model()
    model = Model(Ipopt.Optimizer)

    # Define decision variables
    @variable(model, x[facilities_index, clients_index],Bin)
    @variable(model, y[facilities_index], Bin)

    # # Add constraints
    # @constraint(model, [i in facilities_index], sum( x[i,j] for j in clients_index) >= 1)
    # @constraint(model, [i in facilities_index, j in clients_index], x[i,j] <= y[i])

    # # Set nonlinear objective function
    
    # # facilities_cost = sum(c[i]*y[i] for i in facilities_index) # need to find c
    # facilities_cost = 0
    # equity_cost = sum(sum(x[i,j]*d(i,j) for i in facilities_index) for j in clients_index)
    # # @NLexpression(model, obj_expr, sum(x[i] for i in 1:3))
    # # @NLobjective(model, Min, obj_expr)
    # @objective(model, Min, facilities_cost+equity_cost)
    # return model
end

# Solve the model
model = create_model()
optimize!(model)

# Print results
println("Optimal solution:")
println("x = ", value.(x))
println("Objective value = ", objective_value(model))


LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.ZeroOne are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [58]:
# Define decision variables
model_2 = Model(Ipopt.Optimizer)
@variable(model_2, x[facilities_index, clients_index], binary=true)

LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.ZeroOne are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.